# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/root/miniconda3/envs/yeqing/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/envs/yeqing/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/root/miniconda3/envs/yeqing/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是《易经》中的第一卦，也是八卦之一。乾卦是由两个阴爻夹一个阳爻构成，象征着天、强、积极、刚健等含义。乾卦的卦辞是“元、亨、利、贞”，表示这个卦具有正始、通、顺利、正派的特质。

乾卦的六爻分别有不同的含义。阳爻表示阳刚之气，阴爻表示阴柔之气。在六爻中，阳爻代表阳刚之力量，阴爻代表阴柔之力量。当阳爻在阴爻之中时，代表阳刚之力量受困，但仍有发展的希望。当阴爻在阳爻之中时，代表阴柔之力量受困，但仍有发展的希望。当阳爻在阳爻之中，代表阳刚之力量过于强大，需要有节制。当阴爻在阴爻之中，代表阴柔之力量过于强大，需要有节制。

乾卦的运用，可以指导人们如何应对现实生活中的各种挑战，如何保持内心的刚健和积极。同时，乾卦也提醒人们，在追求成功的过程中，要积极向前，勇往直前，但也要注意保持谦虚和敬畏。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是《易经》中的第二卦，也是八卦之一。讼卦由两个阳爻夹一个阴爻构成，象征着天、刚、争、决等含义。讼卦的卦辞是“无悔、无悔、吉”，表示这个卦具有正无悔恨、吉祥的特质。

讼卦的六爻分别有不同的含义。阳爻表示阳刚之气，阴爻表示阴柔之气。在六爻中，阳爻代表阳刚之力量，阴爻代表阴柔之力量。当阳爻在阴爻之上时，代表阳刚之力量过于强大，需要有节制。当阴爻在阳爻之上时，代表阴柔之力量过于强大，需要有节制。

讼卦的运用，可以指导人们如何应对现实生活中的各种争端和矛盾，如何在争斗中保持理性和公正。同时，讼卦也提醒人们，在处理争端和矛盾时，要充分考虑各方的利益和感受，以求达到公正和平衡。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [10]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp_gpt = "20250817_002513"
timestamp_deepseek = "20250817_140600"

peft_model_path_gpt = f"models/{model_name_or_path}-epoch{epochs}-{timestamp_gpt}"
peft_model_path_deepseek = f"models/{model_name_or_path}-epoch{epochs}-{timestamp_deepseek}"

config_gpt = PeftConfig.from_pretrained(peft_model_path_gpt)
config_deepseek = PeftConfig.from_pretrained(peft_model_path_deepseek)
qlora_model_gpt = PeftModel.from_pretrained(base_model, peft_model_path_gpt)
qlora_model_deepseek = PeftModel.from_pretrained(base_model, peft_model_path_deepseek)
training_gpt=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp_gpt}"
training_deepseek=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp_deepseek}"

In [18]:
def compare_chatglm_results(query, base_model, qlora_model_gpt, qlora_model_deepseek, training_gpt, training_deepseek):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_gpt_out = qlora_model_gpt.generate(**inputs, max_new_tokens=512)
    ft_gpt_response = tokenizer.decode(ft_gpt_out[0], skip_special_tokens=True)

    ft_deepseekt_out = qlora_model_deepseek.generate(**inputs, max_new_tokens=512)
    ft_deepseek_response = tokenizer.decode(ft_deepseekt_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatgpt生成数据微调后（{training_gpt}）：\n{ft_gpt_response}\n\n\nDeepseek生成数据微调后（{training_deepseek}）：\n{ft_deepseek_response}")
    return base_response, ft_gpt_response, ft_deepseek_response

### 微调前后效果对比

In [21]:
base_response, ft_gpt_response, ft_deepseek_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_gpt, qlora_model_deepseek, training_gpt, training_deepseek)

问题：解释下乾卦是什么？

原始输出：
乾卦是八卦之一，它包含了宇宙和生命的活力与力量。在周易中，乾卦作为阳刚之卦，象征着天、云、雨和阳光，以及雄壮和壮丽。卦辞'元亨利贞'表明了创新的开始，亨通的路径，悔恨的避免和永恒的固体。

乾卦的六爻描绘了从 King 到王后，从最高到最低的壮丽景象。六爻的每一爻都包含一个'阳爻'（雄壮、积极）和一个'阴爻'（阴暗、次要）的组合。随着爻的进退，它揭示了阳刚之力的逐步发展，最终达到完善。

乾卦的'天行健'和'君子以自强不息'揭示了人们应该像天一样自强不息，不断前进。卦辞中'利见大人'指示了在困难时刻寻求明智的援助，而'refrain from sin'强调了避免邪恶的重要性。

在现实世界中，乾卦的卦辞和爻辞为人们提供了一个独特的视角，即在追求成功和发展的过程中，如何保持诚实、自强和明智。它教导我们，面对挑战时，应勇敢地追求目标，而当错误发生时，应反思并寻求帮助。乾卦揭示了人生中的困难和挑战，以及如何在困难中保持坚定和积极态度的重要性。


Chatgpt生成数据微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250817_002513）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦作为六十四卦之首，具有至高无上的地位。它由六个阳爻组成，象征纯阳至刚的宇宙本原力量。卦辞'元亨利贞'揭示了天道运行的四大特质：创始（元）、亨通（亨）、有利（利）、正固（贞）。《象传》'天行健，君子以自强不息'的经典阐释，将自然法则与人生修养完美统一。

乾卦以'天'为象，以'龙'为喻，展现了阳刚之气的运行轨迹。从初爻'潜龙勿用'到上爻'亢龙有悔'，完整呈现了事物发展的客观规律。其核心哲学在于：刚健而不失中正，进取而懂得节制。邵雍提醒'谨防太强'，傅佩荣指出'女子过刚宜慎重'，都体现了对阳刚过盛的警示。

在现实应用中，乾卦启示我们：
1. 事业上把握'飞龙在天'的机遇，同时警惕盛极而衰
2. 经商需遵循天道，戒骄戒躁
3. 求名应持续精进，克服自满
4. 婚恋讲究刚柔相济
5. 决策要秉持刚健中正之道

乾卦的现代价值在于：它既激励人们发扬自强不息的精神，又告诫要遵循客观规律，保持忧患意识，实现可持续发展。


Deepseek生成数据微调后（ChatGLM3-6B(Epoch=

In [22]:
base_response, ft_gpt_response, ft_deepseek_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_gpt, qlora_model_deepseek, training_gpt, training_deepseek)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示意味的卦象，由下卦坎（水）和上卦乾（天）组成。这一卦象象征着争讼和冲突，天水相违，流向相反，暗示着事理乖舛和矛盾对立。讼卦的核心哲学是：虽然初期可能有利可图，但最终会带来凶险，因此必须慎之又慎。

讼卦的卦辞指出：“有孚，窒惕，中吉，终凶。”这意味着即使有收获，也要保持警惕，因为事情可能在中期顺利，但最终会走向不利。占得此卦时，适合求助于有德之人，但不宜冒险行动（如涉大川）。

《象辞》进一步解释：“天与水违行，讼。”君子应从中领悟，在做事之初就要谨慎谋划，避免争讼。讼卦提醒人们，争讼非善事，务必戒惧，以和为贵。在事业、经商、求名等方面，此卦均强调退让、隐忍和公正的重要性，避免因逞强或贪图不义之利而招致灾祸。

邵雍解此卦为“天高水深，达远不亲”，寓意人际关系疏离，需谨慎退守。傅佩荣则从时运、财运、家宅等方面指出，此卦宜修身养性，防患于未然。总体而言，讼卦教导人们以柔克刚，知足常乐，避免陷入无谓的争端。


Chatgpt生成数据微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250817_002513）：
[gMASK]sop 周易中的讼卦是什么卦象， 在《周易》体系中，讼卦（坎上坤下）展现了一个充满警示意味的卦象。卦辞'利见大人'暗示着在困境中寻求突破的重要性。卦象中，兑为口（喜悦）、离为火（热情）和震为震的「元吉」等元素组合在一起，表明了积极行动的吉祥效果。

从卦辞到爻辞，都充满了对行动的强调。'利见大人'、'利西南，保其有'、'利见君'等语句，都强调了在困境中寻求突破的重要性。卦象也显示了'老老成成'的规律，暗示成熟稳重的人才能在困境中取得成功。

讼卦的核心哲学是：面对困境时，积极行动是唯一的吉祥之道。卦象中，兑宫（西南）和离宫（东南）都代表着积极的行动方向。同时，卦辞中的'大人'也暗示了需要有能力和智慧才能应对挑战。

在应用于现实生活方面，讼卦强调：当面临困境时，需要积极寻求突破，积极行动，而不是坐等困境消失。卦象显示了积极行动的吉祥效果，同时也提醒：积极行动并非盲目，需要在合适的时机采取合适的方法。


Deepseek生成数据微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-

In [24]:
base_response, ft_gpt_response, ft_deepseek_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model_gpt, qlora_model_deepseek, training_gpt, training_deepseek)

问题：师卦是什么？

原始输出：
在周易中，师卦是一个充满智慧和力量的卦象，由下坎（水）上坤（地）组成。这个卦象象征着智慧与勇敢，具有强烈的创新和发展的趋势。卦辞'利用兵，无仁星'强调战争和竞争的重要性，但同时强调'以多敌少'的原则，暗示通过智慧和谋略能够获得胜利。

卦象分析：上卦为坎，代表水，象征变化和灵活；下卦为坤，代表地，象征稳重和厚实。卦象显示水往地势发展，表示力量和资源的汇聚，暗示发展壮大。卦辞'利用兵，无仁星'表示在战争和竞争中强调智慧与谋略，强调'以智取仁'的原则。

在一般来说，此卦Url格'同姓不欺，福福双全'，强调 Family 的重要性，但'以多敌少'的原则暗示了通过智慧与谋略进行竞争和和发展。


Chatgpt生成数据微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250817_002513）：
[gMASK]sop 师卦是什么？ 在《周易》体系中，师卦（䷆）具有独特而深刻的哲学意蕴。此卦由下坎（水）上坤（地）组成，形成'地中有水'的卦象，象征军事力量蕴藏于民众之中。卦辞'贞，丈人吉，无咎'强调军事行动必须坚守正道，由德高望重的统帅指挥方能吉祥。

从象数角度看，师卦作为坎宫归魂卦，揭示了'忧劳动众，变化无穷'的深层哲理。邵雍的解读指出，得此卦者将面临重重困难，需要'公正无私，排除万难'。傅佩荣则从实用角度提出时运、财运等方面的指导建议。

传统解卦着重分析其军事哲学：虽然'兵凶战危'，但圣人用兵是'不得已而为之'，关键在于'师出有名'和'顺乎形势'。卦象显示养兵聚众之态，暗示激烈的竞争环境。在事业、经商、求名等方面，都强调要刚柔并济、注重合作。

值得注意的是，《象传》'君子以容民畜众'的训诫，将军事智慧升华为治国之道，体现了周易'推天道以明人事'的思维特点。决策方面提示要发挥聪明才智，但需避免过度冒险，以中正之道为根本准则。


Deepseek生成数据微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250817_140600）：
[gMASK]sop 师卦是什么？ 在《周易》体系中，师卦（䷆）具有独特而深刻的哲学意蕴。此卦由下坎（水）上坤（地）组成，形成'地中有水'的卦象，象征军事力量蕴藏于民众之中。卦辞'贞，丈人吉，无咎

## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [14]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [15]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。它反映了天地间阳刚之气的存在，以及天地之间相互信任和相互依赖的关系。

在卜辞中，乾卦预示着天象的运行，以及人类对天象的依赖。同时，它也预示着人类应当刚健强健，积极进取，勇往直前。

在解卦时，取刚健强健之象，象征天之无私，刚健强劲，无所畏惧。在meta-comment中，它提示人们刚健强健，勇往直前，无所畏惧，代表刚健强劲的特性。

在推卦时，则强调刚健强健，柔中带刚，刚中带柔，提示人们刚健强健，勇往直前，无所畏惧。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。它预示着刚健强健的形势，同时也提醒要小心谨慎。根据《象辞》，乾卦象征着刚健强健的特性，同时也要小心谨慎。根据《象辞》，乾卦的卦辞为：“乾，元亨利贞”，这表示它代表天，刚健强健的特性，同时也象征刚健强健的形势。

乾卦表示天，天是刚健强健的，刚强而运行不已。这刚健強健的形势是您所面临的挑战，也是您用来行动的力。根据《辞》，乾卦所象征的刚健强健的特性是：刚健、坚贞、运行不已，这是您所面临挑战时必须具备的品质。

根据《易经》，乾卦的形势是刚健强健的，而所代表的运势是刚健强健的。这表示您在生活中的各种挑战中，需要以刚健强健的毅力来应对，并坚持到底。

总结起来，乾卦是周易中的一卦，代表天，预示着刚健强健的形势，同时也提醒要小心谨慎。它提醒您，在生活中的各种挑战中，要以刚健强健的毅力来应对，并坚持到底。


In [16]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
地水师卦是周易中的一卦，对应着周易的卦辞是“地水师卦：地水（卦）为师（卦）”。这卦的主要含义是，尽管看起来情况并不顺利，但事实上，这是一段美好的历程，因为尽管身处艰难的环境中，但却能保持镇静，并得到周围人的支持。

在占卜中，地水师卦表示事情开始时可能会遇到一些艰难困苦的情况，周围的环境并不顺利，会给人带来压力和挑战。然而，只要准备充分，保持冷静，积极应对，就能化险为夷，达到预期目的。

在解卦时，地形水利，说明顺利，可以得到周围人的帮助。在决策时，应该考虑到事情的实际情况，分析形势，做出明智的选择。

地水师卦的分析需要结合具体的时代背景和情境，不能简单地套用。在不同的情况下， interpreter 可能会给出不同的解释。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？ 地水师卦是周易中的一卦，由上卦坤（地）和下卦坎（水）组成，上卦为地，下卦为水，象征着地中有水，地中有生命的成长过程。根据《象辞》，这一卦象被解释为“地水”，具有“水国”之象。

根据《象辞》，这个卦象表示的是"地中有水"，具有水国之象。这预示了好运 will be reached, and the safe and sound condition will be maintained. According to the commentators in the Book of Divination, the best course of action is to be patient and composed, and to avoid argumentative and confrontational situations.

地水师卦的卦象是地水，这预示了好运 will be reached, and the safe and sound condition will be maintained. According to the commentators in the Book of Divination, the best course of action is to be patient and composed, and to avoid argumentativ

In [17]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
讼卦是周易中的一卦，对应着《易经》中的卦象为乾上坎下。这一卦的核心卦辞是“利见大人”，意味着有利可图，有君子之道。讼卦的核心意义是解决争端，君子应该明辨是非，善于用礼制来平息争端。在卜卦时，利于形成儒林之象，有利于建立人群之间相互协作的关系。

讼卦的哲学基础是：君子应该明辨是非，善用礼制平息争端。在处理争端时，应当时刻保持冷静，注意控制自己的情绪，避免过度冲动。通过冷静分析，找出问题的根源，从而找到解决问题的方法。

讼卦给我们的启示是：在面对争端和冲突时，我们应该保持冷静和理智，通过分析问题，找出根源，从而找到解决问题的方法。同时，我们还需要善于用礼制来平息争端，建立人群之间相互协作的关系。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 周易中的讼卦是什么卦

 讼卦是周易中的一卦，代表诉讼、争斗和诉讼的卦象。它由两个卦象组成，分别为上卦坎（水）和下卦乾（天）。上卦为坎，象征水，表示上卦的特性为险，下卦为乾，象征天，表示下卦的特性为刚。双方相互克制，双方刚强，难以达成共识，因此双方需要斗争，这表示对方刚强，需要刚直之态度。讼卦的运势为吉，但需要刚健刚强，不能慈慈柔顺，否则容易对方刚强刚直，导致诉讼胜利的结论。

讼卦的取义是：水得刚肯，能平能和。天有刚健，不能容留。双方相互克制，需要刚直之态度。

讼卦的启示是：遇到困难和阻力，需要刚健刚强，坚持到底，不要慈慈柔顺，else 刚强刚直，导致诉讼胜利的结论。

讼卦的适宜读音是：讼卦，卦名：坎上乾下，象征天。

讼卦的运势是：凶，需要刚健刚强，不能慈慈柔顺，else 刚强刚直，导致诉讼胜利的结论。

讼卦的解卦是：坤上卦，代表地，象征柔顺。解卦是讼卦的解决方案，双方需要刚健刚强，不能慈慈柔顺，else 刚强刚直，双方容易达成共识，对方也容易退让，因此需要对方，对方容易退让，对方容易退让，对方容易退让。

讼卦的退卦是：乾上卦，代表天，象征刚直。退卦是讼卦的解决方案，对方容易退让，需要对方，对方容易退让，对方容易退让。

讼卦的象数是：天雷震卦（刚直）+ 坎水卦（险）= 刚健刚强

讼卦的运势总结是：对方刚强，需要刚直之态度，否则容易对方刚强刚直，导致诉讼胜利的结论。
